In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
students_df = pd.read_csv("train.csv")

In [ ]:
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

In [ ]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

# Moved the replace_category_for_numbers function definition here for robustness
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

columns_to_onehot =[
    'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)


# Additional preprocessing steps to handle remaining non-numeric columns and 'Miss' values

# 1. Handle 'Miss' in RENDIMIENTO_GLOBAL by dropping rows
onehoted_df = onehoted_df[onehoted_df['RENDIMIENTO_GLOBAL'] != 'Miss']

# Define data_performance_dict here for robustness
data_performance_dict = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
 }

# Apply the category replacement for RENDIMIENTO_GLOBAL BEFORE converting to numeric
onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "RENDIMIENTO_GLOBAL",
    data_performance_dict)

# Ensure RENDIMIENTO_GLOBAL is numeric after cleaning 'Miss' and applying mapping
onehoted_df['RENDIMIENTO_GLOBAL'] = pd.to_numeric(onehoted_df['RENDIMIENTO_GLOBAL'])


# 2. Convert PERIODO_ACADEMICO to numeric
onehoted_df['PERIODO_ACADEMICO'] = pd.to_numeric(onehoted_df['PERIODO_ACADEMICO'], errors='coerce')
# Fill any NaNs that might result from coercion, e.g., with mode or mean
onehoted_df['PERIODO_ACADEMICO'] = onehoted_df['PERIODO_ACADEMICO'].fillna(onehoted_df['PERIODO_ACADEMICO'].mode()[0])

# 3. Handle 'Miss' in INDICADOR_X columns
indicator_cols = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
for col in indicator_cols:
    onehoted_df[col] = pd.to_numeric(onehoted_df[col], errors='coerce') # Convert 'Miss' to NaN
    onehoted_df[col] = onehoted_df[col].fillna(onehoted_df[col].mean()) # Fill NaNs with the mean

# 4. Identify remaining object columns for one-hot encoding
remaining_object_cols = onehoted_df.select_dtypes(include=['object']).columns.tolist()

# Remove 'RENDIMIENTO_GLOBAL' from the list if it's there, as it's the target
if 'RENDIMIENTO_GLOBAL' in remaining_object_cols:
    remaining_object_cols.remove('RENDIMIENTO_GLOBAL')

# Perform one-hot encoding on the remaining categorical columns
onehoted_df = pd.get_dummies(onehoted_df, columns=remaining_object_cols, drop_first=True)

print(onehoted_df.select_dtypes(include=['object']).columns)
onehoted_df.head()

/tmp/ipython-input-766023776.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


Index([], dtype='object')


,PERIODO_ACADEMICO,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,F_TIENEINTERNET_Miss,F_TIENEINTERNET_No,F_TIENEINTERNET_Si,E_PAGOMATRICULAPROPIO_Miss,...,F_EDUCACIONMADRE_Ninguno,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,20212,3,0.322,0.208,0.310,0.267,0,0,1,0,...,False,False,False,True,False,False,False,False,False,False
1,20212,1,0.311,0.215,0.292,0.264,0,1,0,0,...,False,False,False,False,False,False,False,False,False,True
2,20203,1,0.297,0.214,0.305,0.264,0,0,1,0,...,False,False,False,False,False,False,True,False,False,False
3,20195,4,0.485,0.172,0.252,0.190,0,0,1,0,...,False,False,False,False,False,False,True,False,False,False
4,20212,2,0.316,0.232,0.285,0.294,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False


In [ ]:
def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

In [ ]:
y = onehoted_df["RENDIMIENTO_GLOBAL"].values
X = onehoted_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)

Xtr, Xts, ytr, yts = train_test_split(
    X, y,
    test_size=0.2,       # 20% para el conjunto de prueba
    random_state=42      # Para resultados reproducibles
)

print("Datos divididos:")
print(f"- Entrenamiento: X={Xtr.shape}, y={ytr.shape}")
print(f"- Prueba: X={Xts.shape}, y={yts.shape}")

Datos divididos:
- Entrenamiento: X=(554000, 1041), y=(554000,)
- Prueba: X=(138500, 1041), y=(138500,)


In [ ]:
y = onehoted_df["RENDIMIENTO_GLOBAL"].values
X = onehoted_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
# del final_student_df # This line can be removed as final_student_df is not defined
print (X.shape, y.shape)

(106106, 23) (106106,)


In [6]:
# --- Start of code to define students_df and onehoted_df (from previous cells) ---
# From cell -MkH4ISATgsS
students_df = pd.read_csv("train.csv")

# From cell X909AjluT5hM
students_df = students_df.drop(['ID'], axis=1)
students_df = students_df.fillna('Miss')

# From cell yEIfk21YT7z5, including helper functions
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col])
    k = to_onehot(d[col].values)
    r = d.join(pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index), how='outer')
    del(r[col])
    return r

def replace_category_for_numbers(d, col, vals_to_change):
  r = d.copy()
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())
  return r

columns_to_onehot =[
    'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO',
]
onehoted_df = students_df.copy()
for col in columns_to_onehot:
  onehoted_df = replace_column_with_onehot(onehoted_df, col)

# Additional preprocessing steps to handle remaining non-numeric columns and 'Miss' values

# 1. Handle 'Miss' in RENDIMIENTO_GLOBAL by dropping rows
onehoted_df = onehoted_df[onehoted_df['RENDIMIENTO_GLOBAL'] != 'Miss']

# Define data_performance_dict here for robustness
data_performance_dict = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
 }

# Apply the category replacement for RENDIMIENTO_GLOBAL BEFORE converting to numeric
onehoted_df = replace_category_for_numbers(
    onehoted_df,
    "RENDIMIENTO_GLOBAL",
    data_performance_dict)

# Ensure RENDIMIENTO_GLOBAL is numeric after cleaning 'Miss' and applying mapping
onehoted_df['RENDIMIENTO_GLOBAL'] = pd.to_numeric(onehoted_df['RENDIMIENTO_GLOBAL'])


# 2. Convert PERIODO_ACADEMICO to numeric
onehoted_df['PERIODO_ACADEMICO'] = pd.to_numeric(onehoted_df['PERIODO_ACADEMICO'], errors='coerce')
# Fill any NaNs that might result from coercion, e.g., with mode or mean
onehoted_df['PERIODO_ACADEMICO'] = onehoted_df['PERIODO_ACADEMICO'].fillna(onehoted_df['PERIODO_ACADEMICO'].mode()[0])

# 3. Handle 'Miss' in INDICADOR_X columns
indicator_cols = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
for col in indicator_cols:
    onehoted_df[col] = pd.to_numeric(onehoted_df[col], errors='coerce') # Convert 'Miss' to NaN
    onehoted_df[col] = onehoted_df[col].fillna(onehoted_df[col].mean()) # Fill NaNs with the mean

# 4. Identify remaining object columns for one-hot encoding
remaining_object_cols = onehoted_df.select_dtypes(include=['object']).columns.tolist()

# Remove 'RENDIMIENTO_GLOBAL' from the list if it's there, as it's the target
if 'RENDIMIENTO_GLOBAL' in remaining_object_cols:
    remaining_object_cols.remove('RENDIMIENTO_GLOBAL')

# Perform one-hot encoding on the remaining categorical columns
onehoted_df = pd.get_dummies(onehoted_df, columns=remaining_object_cols, drop_first=True)
# --- End of code to define students_df and onehoted_df ---


y = onehoted_df["RENDIMIENTO_GLOBAL"].values
X = onehoted_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)

Xtr, Xts, ytr, yts = train_test_split(
    X, y,
    test_size=0.2,       # 20% para el conjunto de prueba
    random_state=42      # Para resultados reproducibles
)

print("Datos divididos:")
print(f"- Entrenamiento: X={Xtr.shape}, y={ytr.shape}")
print(f"- Prueba: X={Xts.shape}, y={yts.shape}")

# Initialize Linear Regression model
estimator_lr = LinearRegression()

print("Iniciando entrenamiento del modelo de Regresión Lineal...")

estimator_lr.fit(Xtr, ytr)

print("Entrenamiento finalizado.")

y_pred_lr = estimator_lr.predict(Xts)

# Scores de precisión (Accuracy)
train_score_lr = estimator_lr.score(Xtr, ytr)
test_score_lr = estimator_lr.score(Xts, yts)

print(f"\nAccuracy de entrenamiento (In-Sample): {train_score_lr:.4f}")
print(f"Accuracy de prueba (Out-of-Sample): {test_score_lr:.4f}")

/tmp/ipython-input-1001101162.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  r[col] = r[col].replace(vals_to_change.keys(), vals_to_change.values())


Datos divididos:
- Entrenamiento: X=(554000, 1041), y=(554000,)
- Prueba: X=(138500, 1041), y=(138500,)
Iniciando entrenamiento del modelo de Regresión Lineal...
Entrenamiento finalizado.

Accuracy de entrenamiento (In-Sample): 0.2812
Accuracy de prueba (Out-of-Sample): 0.2816
